# Práctica 4: Redes Neuronales Convolucionales

Importa las librerías que vayas a necesitar:

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

## Seleccionar dataset e importarlo

Busca en la web de Kaggle (o en cualquier otra) un dataset de cualquier tema que te interese. Procura que no sea excesivamente grande para evitar un tiempo de entrenamiento mayor:

In [9]:
logos_train = tf.keras.utils.image_dataset_from_directory(directory="04_Data/Car_Brand_Logos/Train/")
logos_val = tf.keras.utils.image_dataset_from_directory(directory="04_Data/Car_Brand_Logos/Test/")

Found 2441 files belonging to 8 classes.
Found 390 files belonging to 8 classes.


## Exploramos el dataset

Exploramos el formato del dataset: forma, número de elementos, número de "clases" finales...

In [10]:
logos_train.class_names

['hyundai',
 'lexus',
 'mazda',
 'mercedes',
 'opel',
 'skoda',
 'toyota',
 'volkswagen']

## Preprocesamiento del dataset

Haz las operaciones oportunas sobre el set de datos (en caso de ser necesarias). Por ejemplo, si se trata de imágenes en color y quieres trabajar en blanco y negro cambia sus dimensiones. O si son valores entre 0 y 255 escalalos al intervalo 0-1:

In [11]:
num_classes = 8
num_epochs = 25

## Construir el Modelo

Construir la red neuronal requiere configurar las capas del modelo y luego compilar el modelo. Para ello, recuerda que puedes revisar el ejemplo realizado por Marcos y Carmen (Conv2D) o Ainhoa y Carlos (Conv3D) en la práctica 2, además de tener en cuenta los siguientes aspectos importantes:

*   Según nos adentramos en capas más profundas, el tamaño de las imágenes se debería ir reescalando por uno menor (utilizando la **capa MaxPool**)
*   Según nos adentramos en capas más profundas, la cantidad de capas convolucionales seguidas ha de ser mayor para que detecten patrones más complejos.
*   El final de la red será una **capa Flatten** para "aplanar" las imágenes resultantes de capas anteriores, seguido de tantas **capas Dense** como necesites.
*   Recuerda dejar en la última capa densa tantas neuronas como "clases" tengas que diferenciar.

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(4, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(16, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(num_classes)
])

### Compilamos el modelo

Antes de que el modelo este listo para entrenar , se necesitan algunas configuraciones mas: el optimizador, la función de coste o las métricas a utilizar:

In [13]:
model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])


## Entrenar el Modelo

In [14]:
model.fit(
    logos_train,
    validation_data=logos_val,
    epochs=num_epochs
)

Epoch 1/25
77/77 [==============================] - 31s 387ms/step - loss: 2.0690 - accuracy: 0.1585 - val_loss: 2.0276 - val_accuracy: 0.1897
Epoch 2/25
77/77 [==============================] - 30s 376ms/step - loss: 1.8077 - accuracy: 0.3314 - val_loss: 1.6000 - val_accuracy: 0.4487
Epoch 3/25
77/77 [==============================] - 28s 356ms/step - loss: 1.4096 - accuracy: 0.5272 - val_loss: 1.4864 - val_accuracy: 0.5513
Epoch 4/25
77/77 [==============================] - 28s 356ms/step - loss: 1.1505 - accuracy: 0.6104 - val_loss: 1.5540 - val_accuracy: 0.5436
Epoch 5/25
77/77 [==============================] - 28s 353ms/step - loss: 0.9399 - accuracy: 0.6903 - val_loss: 1.7019 - val_accuracy: 0.5667
Epoch 6/25
77/77 [==============================] - 28s 357ms/step - loss: 0.7475 - accuracy: 0.7456 - val_loss: 1.4038 - val_accuracy: 0.6154
Epoch 7/25
77/77 [==============================] - 28s 356ms/step - loss: 0.6118 - accuracy: 0.7923 - val_loss: 1.4419 - val_accuracy: 0.6256

## Evaluar Exactitud

Comparar el rendimiento del modelo sobre el dataset de test:

In [15]:
model.evaluate(logos_val)

13/13 [==============================] - 2s 116ms/step - loss: 3.5338 - accuracy: 0.6538


[3.5337958335876465, 0.6538461446762085]

## Hacer predicciones

Con el modelo entrenado probar a hacer predicciones sobre alguna imagen concreta:

Código para comprobar si el formato de las imágenes es correcto

In [16]:
import os
import imghdr

def check_images( s_dir, ext_list):
    bad_images=[]
    bad_ext=[]
    s_list= os.listdir(s_dir)
    for klass in s_list:
        klass_path=os.path.join (s_dir, klass)
        print ('processing class directory ', klass)
        if os.path.isdir(klass_path):
            file_list=os.listdir(klass_path)
            for f in file_list:               
                f_path=os.path.join (klass_path,f)
                tip = imghdr.what(f_path)
                if ext_list.count(tip) == 0:
                    bad_images.append(f_path)
                if os.path.isfile(f_path):
                    try:
                        img=plt.imread(f_path)
                        shape=img.shape
                    except:
                        print('file ', f_path, ' is not a valid image file')
                        bad_images.append(f_path)
                else:
                    print('*** fatal error, you a sub directory ', f, ' in class directory ', klass)
        else:
            print ('*** WARNING*** you have files in ', s_dir, ' it should only contain sub directories')
    return bad_images, bad_ext

source_dir = "04_Data/Car_Brand_Logos/Train"
good_exts=['jpg', 'png', 'jpeg', 'gif', 'bmp' ] # list of acceptable extensions
bad_file_list, bad_ext_list=check_images(source_dir, good_exts)
if len(bad_file_list) !=0:
    print('improper image files are listed below')
    for i in range (len(bad_file_list)):
        print (bad_file_list[i])
else:
    print(' no improper image files were found')

processing class directory  hyundai
processing class directory  lexus
processing class directory  mazda
processing class directory  mercedes
processing class directory  opel
processing class directory  skoda
processing class directory  toyota
processing class directory  volkswagen
 no improper image files were found
